# Quantum Tunnelling

In [ ]:
using LinearAlgebra
using SparseArrays
using GLMakie

#  Parameters 
ħ = 1.0        # Reduced Planck constant
m = 1.0        # Particle mass
x_min, x_max = -10.0, 10.0  # space domain 
N = 500                    # Number of spatial points
x = range(x_min, x_max, length=N)
dx = step(x)

# Time parameters
t_min, t_max = 0.0, 1    # simulation time
dt = 0.0001
ts = t_min:dt:t_max

#  Potential Barrier 
V0 = 170.0                  # Barrier height
barrier_width = 1
barrier_center = 0.0

function V_potential(x)
    if abs(x - barrier_center) < barrier_width/2
        return V0
    else
        return 0.0
    end
end

V_array = [V_potential(xi) for xi in x]

# Single Wave Packet Moving Left to Right 
x0 = -5.0                  # Initial position
k0 = 18.0                  # Positive momentum (moving right)
σ = 0.5                  # Width of the Gaussian

function ψ0(x)
    return exp(- (x - x0)^2 / (2 * σ^2)) * exp(im * k0 * x)
end


ψ_initial = [ψ0(xi) for xi in x]
norm_factor = sqrt(sum(abs2.(ψ_initial)) * dx)
ψ = ψ_initial / norm_factor


r = im * ħ * dt / (4 * m * dx^2)


main_diag = zeros(ComplexF64, N)
off_diag = fill(-r, N-1)

for i in 1:N
    main_diag[i] = 1 + 2r + im * dt * V_array[i] / (2 * ħ)
end

A = spdiagm(-1 => off_diag, 0 => main_diag, 1 => off_diag)

main_diag_B = zeros(ComplexF64, N)
for i in 1:N
    main_diag_B[i] = 1 - 2r - im * dt * V_array[i] / (2 * ħ)
end
B = spdiagm(-1 => -off_diag, 0 => main_diag_B, 1 => -off_diag)


A[1, :] .= 0; A[1, 1] = 1
A[end, :] .= 0; A[end, end] = 1
B[1, :] .= 0; B[1, 1] = 1
B[end, :] .= 0; B[end, end] = 1


ψsave = zeros(ComplexF64, N, length(ts))
ψsave[:, 1] = ψ


A_factor = factorize(A)  

for j in 2:length(ts)
    b = B * ψ
    ψ = A_factor \ b
    ψsave[:, j] = ψ
end

barrier_left = barrier_center - barrier_width/2
barrier_right = barrier_center + barrier_width/2


left_region = x .< barrier_left
barrier_region = (x .>= barrier_left) .& (x .<= barrier_right)
right_region = x .> barrier_right

trans_probs = zeros(length(ts))
refl_probs = zeros(length(ts))
barrier_probs = zeros(length(ts))  

for i in 1:length(ts)
    prob_density = abs2.(ψsave[:, i])
    total_prob = sum(prob_density) * dx  
    refl_probs[i] = sum(prob_density[left_region]) * dx / total_prob
    trans_probs[i] = sum(prob_density[right_region]) * dx / total_prob
    barrier_probs[i] = sum(prob_density[barrier_region]) * dx / total_prob
end

# Visualization
fig = Figure(resolution=(1000, 700))
ax = Axis(fig[1, 1], 
    xlabel="Position (x)", 
    ylabel="Probability Density |ψ|²", 
    title="Quantum Tunneling: t = 0.0000 | Refl: 0.0% | Trans: 0.0%"
)


x_obs = Observable(x)
y_obs = Observable(abs2.(ψsave[:, 1]))
v_obs = Observable(V_array ./ maximum(V_array) .* maximum(abs2.(ψsave)))

wf_line = lines!(ax, x_obs, y_obs, color=:blue, linewidth=1.5, label="|ψ|²")

barrier_line = lines!(ax, x_obs, v_obs, 
    color=:red, linestyle=:dash, linewidth=2, label="Potential Barrier")

vlines!(ax, [barrier_left, barrier_right], 
        color=:gray, linestyle=:dot, linewidth=1)

xlims!(ax, x_min, x_max)
ylims!(ax, 0, maximum(abs2.(ψsave)) * 1.1)

axislegend(ax, position=:rt)

framerate = 30
record(fig, "quantum_tunnelling.mp4", 1:10:length(ts); framerate=framerate) do i
    y_obs[] = abs2.(ψsave[:, i])
    
    ax.title[] = "Quantum Tunneling: t = $(round(ts[i], digits=3)) | " *
                 "Refl: $(round(refl_probs[i]*100, digits=1))% | " *
                 "Trans: $(round(trans_probs[i]*100, digits=1))%" *
                 "Barrier: $(round(barrier_probs[i]*100, digits=1))% | " *
                 "Total: $(round((refl_probs[i] + trans_probs[i] + barrier_probs[i])*100, digits=1))%"
end

